### Basic Imports and scraper instantiation

In [1]:
from Scrapers.journalscrapers import BioMedCentralScraper, ElsevierScraper, ExistingScraper, \
    HindawiScraper, PLOSScraper, SageHybridScraper, SpringerHybridScraper, SpringerOpenScraper, \
    WileyScraper

import json
import csv
"""scrapers = [
    BioMedCentralScraper("https://www.biomedcentral.com/journals"),
    ElsevierScraper("data/elsevier/2016-uncleaned.csv"),
    ExistingScraper("data/OA_journals.tsv"),
    HindawiScraper("http://www.hindawi.com/apc/"),
    PLOSScraper("https://www.plos.org/publication-fees"),
    SageHybridScraper(""),
    SpringerHybridScraper("data/springer/2016+Springer+Journals+List.csv"),
    SpringerOpenScraper("http://www.springeropen.com/journals"),
    WileyScraper("http://olabout.wiley.com/WileyCDA/Section/id-828038.html")
]"""


'scrapers = [\n    BioMedCentralScraper("https://www.biomedcentral.com/journals"),\n    ElsevierScraper("data/elsevier/2016-uncleaned.csv"),\n    ExistingScraper("data/OA_journals.tsv"),\n    HindawiScraper("http://www.hindawi.com/apc/"),\n    PLOSScraper("https://www.plos.org/publication-fees"),\n    SageHybridScraper(""),\n    SpringerHybridScraper("data/springer/2016+Springer+Journals+List.csv"),\n    SpringerOpenScraper("http://www.springeropen.com/journals"),\n    WileyScraper("http://olabout.wiley.com/WileyCDA/Section/id-828038.html")\n]'

### Writing output of scrapers to file for saving

In [ ]:
with open("data/OA_journal_prices_2016.csv", "w") as f:
    for scraper in scrapers:
        writer = csv.writer(f)
        writer.writerow(["publisher_name", "journal_name", "date_stamp", "is_hybrid", "issn", "apc"])
        try:
            for i in scraper.get_entries():
                writer.writerow([j.encode('utf8') if type(j) != bool else j for j in i])
        except StopIteration:
            print str(scraper) + " isn't implemented yet."

### Generating Web Requests

In [ ]:
from journalscrapers import BioMedCentralScraper, ElsevierScraper, ExistingScraper, \
    HindawiScraper, PLOSScraper, SageHybridScraper, SpringerHybridScraper, SpringerOpenScraper, \
    WileyScraper

import json

scrapers = [
    #BioMedCentralScraper("https://www.biomedcentral.com/journals"),
    #ElsevierScraper("data/elsevier/2016-uncleaned.csv"),
    #ExistingScraper("data/OA_journals.tsv"),
    #HindawiScraper("http://www.hindawi.com/apc/"),
    #PLOSScraper("https://www.plos.org/publication-fees"),
    #SageHybridScraper(""),
    #SpringerHybridScraper("data/springer/2016+Springer+Journals+List.csv"),
    #SpringerOpenScraper("http://www.springeropen.com/journals"),
    #WileyScraper("http://olabout.wiley.com/WileyCDA/Section/id-828038.html")
]

agg_journal_requests = []
agg_price_requests = []
for scraper in scrapers:
    try:
        for i in scraper.get_entries():
            raw_data = dict(zip(["pub", "name", "date_stamp", "is_hybrid", "issn", "apc"], i))
            print raw_data
            agg_journal_requests.append({
                'issn': raw_data['issn'],
                'journal_name': raw_data['name'],
                'pub_name': raw_data['pub'],
                'is_hybrid': raw_data['is_hybrid'],
            })

            agg_price_requests.append({
                'issn': raw_data['issn'],
                'price': raw_data['apc'],
                'date_stamp': raw_data['date_stamp'],
            })
    except StopIteration:
        print str(scraper) + " isn't implemented yet."


with open("data/journal_requests.json", "a+") as f:
    f.write(json.dumps(agg_journal_requests))

with open("data/price_requests.json", "r") as f:
    pl = json.load(f)

with open("data/price_requests.json", "w") as f:
    f.write(json.dumps(pl + agg_price_requests))

### Adding the Artricle Influence Requests

### Importing requests into the main site

In [ ]:
url = 'http://52.175.212.92/'

In [ ]:
url = 'http://localhost:8000/'

In [ ]:
import requests
import json

r = requests.post(url + 'api-token-auth/',
                  data={
                      'username': '',
                      'password': '',
                  })
token = "Token " + json.loads(r.text)['token']
print token

request_dicts = [
    #{"file_path": "category_update_requests.json", "url_stub": "api/journals/"},
    #{"file_path": "journal_requests.json", "url_stub": "api/journals/"},
    #{"file_path": "influence_requests.json", "url_stub": "api/influence/"},
    {"file_path": "price_requests.json", "url_stub": "api/prices/"},
    ]
valid_issn = set()
for rd in request_dicts:
    with open("data/" + rd['file_path'], "r") as f:
        for request_data in json.load(f):
            if rd['file_path'] == "journal_requests.json":
                valid_issn.add(request_data['issn'])
            #elif request_data['issn'] in valid_issn:
            if rd['file_path'] != "influence_requests.json" or request_data['issn'] in valid_issn:
                #and rd['file_path'] != "journal_requests.json":
                request = requests.put(
                    url + rd['url_stub'] + request_data['issn'] + "/",
                    headers={
                        'Authorization': token,
                        'Content-Type': 'application/json',
                    },
                    data=json.dumps(request_data),
                )
                print request_data
                print request.status_code
            else:
                pass


### Fixing Categories

In [ ]:
import requests
import json
import csv


web_url = 'http://52.175.212.92/'
dev_url = 'http://localhost:8000/'
url = web_url

r = requests.post(url + 'api-token-auth/',
                  data={
                      'username': '',
                      'password': ''
                  })
token = "Token " + json.loads(r.text)['token']
print token
with open("data/journal_requests.json", "r") as f:
    valid_issns = {journal['issn'] for journal in json.load(f)}

with open("data/journals_EF_AI_2014.txt", "r") as f:
    client = requests.session()

    for journal in csv.reader(f, dialect=csv.excel_tab):
        if journal[1] in valid_issns and journal[23] != "NULL":
            request_data = {"issn": journal[1], "category": journal[23]}

            request = requests.put(
                url + "api/journals/" + journal[1] + "/",
                headers={
                    'Authorization': token,
                    'Content-Type': 'application/json',
                },
                data=json.dumps(request_data),
            )
            print request.status_code



### Fixing the elsevier stuff in the data base

In [ ]:
from journalscrapers import ElsevierScraper
import requests
import json

web_url = 'http://52.175.212.92/'
dev_url = 'http://localhost:8000/'
url = web_url
r = requests.post(url + 'api-token-auth/',
                  data={
                      'username': '',
                      'password': ''
                  })
token = "Token " + json.loads(r.text)['token']
print token
scraper = ElsevierScraper("data/elsevier/2016-uncleaned.csv")

agg_journal_requests = []
agg_price_requests = []
try:
    for i in scraper.get_entries():
        raw_data = dict(zip(["pub", "name", "date_stamp", "is_hybrid", "issn", "apc"], i))

        request_data = {
            'issn': raw_data['issn'],
            'price': raw_data['apc'],
            'date_stamp': raw_data['date_stamp'],
        }
        request = requests.put(
            url + "api/prices/" + request_data['issn'] + "/",
            headers={
                'Authorization': token,
                'Content-Type': 'application/json',
            },
            data=json.dumps(request_data),
        )
        print request.status_code
except StopIteration:
    print str(scraper) + " isn't implemented yet."

